In [2]:
import pandas as pd
import numpy as np
import os
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [3]:
cliente_tabla = pd.read_csv('../data/cliente_tabla.csv')
producto_tabla = pd.read_csv('../data/producto_tabla.csv')
test = pd.read_csv('../data/test.csv')
train = pd.read_csv('../data/train.csv')
town_state = pd.read_csv('../data/town_state.csv')

# Group by Client_ID information


- most common route
- most common agencia id
- all routes
- all agencies
- how many routes
- how many agencies

In [17]:
test_train = pd.concat([train[['Cliente_ID', 'Agencia_ID', 'Producto_ID', 'Semana', 'Ruta_SAK']],
                       test[['Cliente_ID', 'Agencia_ID', 'Producto_ID', 'Semana', 'Ruta_SAK']]], axis = 0)
test_train = pd.merge(test_train, town_state, left_on='Agencia_ID', right_on='Agencia_ID', how='left' )
test_train.sample(5)

,Cliente_ID,Agencia_ID,Producto_ID,Semana,Ruta_SAK,Town,State
76660010,153256,1693,5355,10,3919,2377 VERACRUZ SUR MLA,VERACRUZ
23491662,216247,1212,35651,5,1427,2064 AG. TEPALCATES 1,"MÉXICO, D.F."
41146452,1818037,2216,41938,6,1151,2380 CAMPECHE,CAMPECHE
19830748,385654,2060,32940,4,2152,2301 CUBILETE,GUANAJUATO
18141217,145268,1952,32940,4,5722,2305 MAZATLAN BIMBO,SINALOA


In [15]:
town_per_client = test_train[['Town', 'Cliente_ID']].groupby('Cliente_ID').agg(lambda x:x.value_counts().index[0])
town_per_client.head()

,Town
Cliente_ID,
26,2175 TOLUCA AEROPUERTO
60,2175 TOLUCA AEROPUERTO
65,2175 TOLUCA AEROPUERTO
101,2655 LOS MOCHIS
105,2175 TOLUCA AEROPUERTO


In [28]:
ruta_per_client = test_train[['Ruta_SAK','Cliente_ID']].groupby('Cliente_ID').agg(lambda x:x.value_counts().index[0])
all_ruta_per_client = test_train[['Ruta_SAK','Cliente_ID']].groupby('Cliente_ID').agg(lambda x:str(np.unique(x)))
all_ruta_per_client = all_ruta_per_client.rename(columns = {'Ruta_SAK':'All_Rutas'})
N_ruta_per_client = test_train[['Ruta_SAK','Cliente_ID']].groupby('Cliente_ID').agg(lambda x:len(np.unique(x)))
N_ruta_per_client = N_ruta_per_client.rename(columns = {'Ruta_SAK':'N_rutas'})

In [29]:
agencia_per_client = test_train[['Agencia_ID','Cliente_ID']].groupby('Cliente_ID').agg(lambda x:x.value_counts().index[0])
all_agencia_per_client = test_train[['Agencia_ID','Cliente_ID']].groupby('Cliente_ID').agg(lambda x:str(np.unique(x)))
all_agencia_per_client = all_agencia_per_client.rename(columns = {'Agencia_ID':'All_Agencias'})
N_agencia_per_client = test_train[['Agencia_ID','Cliente_ID']].groupby('Cliente_ID').agg(lambda x:len(np.unique(x)))
N_agencia_per_client = N_agencia_per_client.rename(columns = {'Agencia_ID':'N_Agencias'})

In [40]:
client_info = pd.merge(town_per_client, ruta_per_client, how='left', right_index=True, left_index=True)
client_info = pd.merge(client_info, agencia_per_client, how='left', right_index=True, left_index=True)
client_info = pd.merge(client_info, all_ruta_per_client, how='left', right_index=True, left_index=True)
client_info = pd.merge(client_info, all_agencia_per_client, how='left', right_index=True, left_index=True)
client_info = pd.merge(client_info, N_ruta_per_client, how='left', right_index=True, left_index=True)
client_info = pd.merge(client_info, N_agencia_per_client, how='left', right_index=True, left_index=True)
client_info['Cliente_ID'] = client_info.index
client_info = client_info.reset_index(drop=True)
client_info.head()

,Town,Ruta_SAK,Agencia_ID,All_Rutas,All_Agencias,N_rutas,N_Agencias
Cliente_ID,,,,,,,
26,2175 TOLUCA AEROPUERTO,7212,2061,[3076 3079 3092 4189 7212 9308],[2061 2655],6,2
60,2175 TOLUCA AEROPUERTO,7238,2061,[3025 4120 4121 4122 4124 4125 4126 4127 4128 ...,[2061 2655],22,2
65,2175 TOLUCA AEROPUERTO,7250,2061,[7250 7333 9033],[2061 2655],3,2
101,2655 LOS MOCHIS,7334,2655,[7334],[2655],1,1
105,2175 TOLUCA AEROPUERTO,7222,2061,[7222],[2061],1,1


In [44]:
client_info.to_csv('data/Derived/client_info.csv', index = False)

# Pivot table in wide format

In [5]:
train.head()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.14,0,0,3
1,3,1110,7,3301,15766,1216,4,33.52,0,0,4
2,3,1110,7,3301,15766,1238,4,39.32,0,0,4
3,3,1110,7,3301,15766,1240,4,33.52,0,0,4
4,3,1110,7,3301,15766,1242,3,22.92,0,0,3


In [6]:
test_temp = test
test_temp['Demanda_uni_equil'] = None
test_train = pd.concat([train[['Cliente_ID',  'Producto_ID', 'Semana','Demanda_uni_equil' ]],
                       test_temp[['Cliente_ID', 'Producto_ID', 'Semana','Demanda_uni_equil' ]]], axis = 0)
test_train.sample(5)

,Cliente_ID,Producto_ID,Semana,Demanda_uni_equil
17778183,102264,43084,4,2
64438148,1519852,1182,9,8
30685404,1138467,30532,5,23
11619674,173705,73,4,1
65117220,1185743,5310,9,8


In [7]:
pivot_test_train = pd.pivot_table(data = test_train, values = 'Demanda_uni_equil', index = ['Cliente_ID', 'Producto_ID'],columns='Semana' )
pivot_test_train.columns = ['sem%i'%i for i in pivot_test_train.columns]
pivot_test_train = pivot_test_train.reset_index()
pivot_test_train.head()

In [15]:
pivot_test_train.to_csv('../data/Derived/pivot_test_train.csv', index = False)